In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
# from wrangle import box_plot, hist_plot, wrangle_zillow
# from prepare import scale_data, visualize_scaler
# from explore import blah, blah, blah

In [ ]:
SELECT * FROM properties_2017
JOIN propertylandusetype
USING(propertylandusetypeid)
JOIN predictions_2017
USING(parcelid)
LIMIT 100;